# AOC2019 - Day 15

Today's challenge: [Day 15](https://adventofcode.com/2019/day/15)

Data are manually downloaded into `./data/input_15.txt`

In [1]:
with open('./data/input_15.txt') as f:
    raw = f.read()

In [2]:
from Intcode import Intcode

from IPython import display
from time import sleep

In [3]:
from collections import defaultdict
data = [int(x) for x in raw.strip().split(",")]

In [4]:
class RC:
    def __init__(self, brain):
        self.p = brain
        self.x = 0
        self.y = 0
        self.panel = defaultdict(lambda : " ")
        self.panel[(0,0)] = 'o'
        self.start = (0,0)
        self.target = (None, None)
        self.d_ = 0 # wlog start north
        self.d = 1
        self.directions = [1,4,2,3] # N, E, S, W, clockwise
        
    def __repr__(self):
        xs, ys = zip(*self.panel.keys())
        x0, x1 = min(xs), max(xs)
        y0, y1 = min(ys), max(ys)
        res = []
        for y in range(y0, y1+1):
            row = ""
            for x in range(x0,x1+1):
                if (x,y) == self.start:
                    row += 'S'
                elif (x,y) == self.target:
                    row += 'T'
                else:
                    row += self.panel[(x,y)]
            res.append(row)
        return "\n".join(res)
    
    def rotate(self, a):
        self.d_ = (self.d_ + 4 + a) % 4
        self.d = self.directions[self.d_]
        
    def move(self, next_move):
        # remove myself from old position
        self.panel[(self.x, self.y)] = '.'
        if next_move == 1: #N
            self.y -= 1
        elif next_move == 2: #S
            self.y += 1
        elif next_move == 3: #W
            self.x -= 1
        elif next_move == 4: #E
            self.x += 1
        # now, add myself in new position
        self.panel[(self.x, self.y)] = 'o'
    
    def draw_wall(self, nm):
        if nm == 1: #N
            self.panel[(self.x, self.y-1)] = '#'
        elif nm == 2: #S
            self.panel[(self.x, self.y+1)] = '#'
        elif nm == 3: #W
            self.panel[(self.x-1, self.y)] = '#'
        elif nm == 4: #E
            self.panel[(self.x+1, self.y)] = '#'
                
    def run(self, s=1):
        while True:
            next_move = self.d
            out = self.p.run(next_move)
            if out == 0: # hit a wall
                self.draw_wall(next_move)
                self.rotate(-1)
            else: # proceed
                self.move(next_move)
                if out == 2:
                    self.target = (self.x, self.y)
                if self.x == 0 and self.y == 0:
                    break
                self.rotate(1)
            print(self)
            display.clear_output(wait=True)
            sleep(s)


In [5]:
p = Intcode(data[:])
remote = RC(p)
remote.run(s=0.001)

 ########### ######### ######### ####### 
#...........#.........#.........#.......#
#.#####.#####.#######.###.#.#####.###.#.#
#.#.....#.....#.....#...#.#.........#.#.#
#.#.#####.#######.#.###.#.#########.#.## 
#.#.#.....#.#.....#.#...#.....#...#.#...#
#.###.#####.#.#.#####.#########.#.#####.#
#.#...#.#...#.#.....#.....#.....#.......#
#.#.###.#.#.#.#####.#####.#.#####.#####.#
#.#.#T#...#.#.#...#.#.....#.....#.#...#.#
#.#.#.#.###.#.###.#.#.#####.###.###.#.#.#
#...#.....#.......#.#.#.......#.....#.#.#
#.###.#############.#.#######.#######.#.#
#.#...#...........#.#.....#.#.....#...#.#
#.###.#.#.#######.#.#####.#.###.###.###.#
#...#.#.#.#...#.#...#.....#.....#...#.#.#
 ##.#.#.###.#.#.###.#.#######.###.###.#.#
#...#.#.....#.#.....#...#...#.#...#...#.#
#.#####.#####.#######.#.#.#.#.#.#####.#.#
#.....#.#.....#.....#.#...#.#.#.#.....#.#
 ####.#.#.#.###.###.###.###.###.#.#####.#
#.#...#.#.#.#...#.#.oS#.#.#...#.#.#.....#
#.#.#####.###.###.#####.#.###.#.#.#.#### 
#.#.......#...#...#.....#.#...#.#.

In [6]:
import networkx as nx

In [7]:
lab = str(remote).split("\n")
g_lab = nx.Graph()

for y, row in enumerate(lab[:-1]):
    for x, c in enumerate(row[:-1]):
        if c == '#' or c == ' ':
            continue
        if c == 'T':
            T = (x, y)
        if c == 'S':
            S = (x, y)
        if row[x+1] in ['S', 'T', '.', 'o']:
            g_lab.add_edge((x,y), (x+1,y))
        if lab[y+1][x] in ['S', 'T', '.', 'o']:
            g_lab.add_edge((x,y), (x,y+1))

In [8]:
# part 1
nx.shortest_path_length(g_lab, T, S)

240

In [9]:
# part 2
max(nx.shortest_path_length(g_lab, T).values())

322